<a href="https://colab.research.google.com/github/adam-bozman/rapidfuzz/blob/main/RapidFuzz_DealerName.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A Simple Rapidfuzz Example

## Imports

In [107]:
####################
# Import Libraries #
####################

import pandas as pd
import numpy as np
import unicodedata
from unicodedata import normalize
import re
import nltk
from nltk.corpus import stopwords
import unicodedata

In [108]:
!pip install rapidfuzz -q

In [109]:
from rapidfuzz import fuzz
from rapidfuzz import process

In [110]:
# Import CSV (or XLSX)

# Selecting only the first column
# This CSV in particular appears to have been encoded on MAC OS initially
# If not the case, ignore 'mac_roman'

df1 = pd.read_csv(r"/content/dealer_check.csv", usecols=['dealer_naic'], encoding='mac_roman')
df1.rename(columns ={'dealer_naic':'names'}, inplace = True) 
# df1 = df1.iloc[:501, ]

df2 = pd.read_csv(r"/content/dealer_check.csv", usecols=['dealer_hist'], encoding='mac_roman')
df2.rename(columns ={'dealer_hist':'dealers'}, inplace = True) 
df2 = df2.iloc[:423, ]

## Preprocessing Data

In [111]:
df1['names'] = df1['names'].fillna("")

df1['names']=df1['names'].str.lower()

def remove_whitespace(text):
    return  " ".join(text.split())
df1['names']=df1['names'].apply(remove_whitespace)

import nltk
nltk.download('punkt')
from nltk import word_tokenize

df1['names']=df1['names'].apply(lambda X: word_tokenize(X))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [112]:
nltk.download('stopwords')
from nltk.corpus import stopwords

en_stopwords = stopwords.words('english')

def remove_stopwords(text):
    result = []
    for token in text:
        if token not in en_stopwords:
            result.append(token)
            
    return result
  
df1['names'] = df1['names'].apply(remove_stopwords)

from nltk.tokenize import RegexpTokenizer

def remove_punct(text):
    
    tokenizer = RegexpTokenizer(r"\w+")
    lst=tokenizer.tokenize(' '.join(text))
    return lst

df1['names'] = df1['names'].apply(remove_punct)

nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize,pos_tag

def lemmatization(text):
    
    result=[]
    wordnet = WordNetLemmatizer()
    for token,tag in pos_tag(text):
        pos=tag[0].lower()
        
        if pos not in ['a', 'r', 'n', 'v']:
            pos='n'
            
        result.append(wordnet.lemmatize(token,pos))
    
    return result

df1['names']=df1['names'].apply(lemmatization)

from nltk.stem import PorterStemmer

def stemming(text):
    porter = PorterStemmer()
    
    result=[]
    for word in text:
        result.append(porter.stem(word))
    return result

df1['names']=df1['names'].apply(stemming)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [113]:
import re
def remove_tag(text):
    
    text=' '.join(text)
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)
df1['names'] = df1['names'].apply(remove_tag)

def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', str(text))

df1['names'] = df1['names'].apply(remove_urls)

In [114]:
def remove_nums(text):
    nums_pattern = re.compile(r'[0-9]')
    return nums_pattern.sub(r'', str(text))

df1['names'] = df1['names'].apply(remove_nums)

In [115]:
dframe1 = df1
dframe2 = df2

## The Fuzzy Process

In [116]:
# empty lists for storing the matches
# later
mat1 = []
mat2 = []
p = []
  
# printing the pandas dataframes
print("First dataframe:\n", dframe1,
      "\nSecond dataframe:\n", dframe2)
  
# converting dataframe column 
# to list of elements
# to do fuzzy matching
list1 = dframe1['names'].tolist()
list2 = dframe2['dealers'].tolist()
  
# taking the threshold as 60
threshold = 60
  
# iterating through list1 to extract
# it's closest match from list2
for i in list1:
    mat1.append(process.extractOne(
      i, list2, scorer=fuzz.token_set_ratio))
dframe1['matches'] = mat1
  
# iterating through the closest matches
# to filter out the maximum closest match
for j in dframe1['matches']:
    if j[1] >= threshold:
        p.append(j[0])
    mat2.append(",".join(p))
    p = []
  
  
# storing the resultant matches back 
# to dframe1
dframe1['matches'] = mat2
print("\nDataFrame after Fuzzy matching using token_set_ratio():")
dframe1

First dataframe:
                                names
0               arbor research trade
1                              stern
2                    goldman sach co
3                       sandler neil
4      smith barney harri co correct
...                              ...
25383                    zurinsco zi
25384                          zwirn
25385                     zzz broker
25386                           _gsl
25387             spear leed kellogg

[25388 rows x 1 columns] 
Second dataframe:
                                        dealers
0         ABN AMRO BANK, N.V., NY BR          
1         ABN AMRO INCORPORATED               
2          AUBREY G. LANSTON & CO., INC.      
3          BA SECURITIES, INC.                
4    BANC OF AMERICA SECURITIES LLC           
..                                         ...
418                       SALOMON SMITH BARENY
419                       SOLOMAN SMITH BARNEY
420                       USCC/BANCO SANTANDER
421                   

,names,matches
0,arbor research trade,ARBOR RESEARCH&TRADING
1,stern,STERNE
2,goldman sach co,"GOLDMAN, SACHS & CO."
3,sandler neil,
4,smith barney harri co correct,SMITH BARNEY
...,...,...
25383,zurinsco zi,
25384,zwirn,
25385,zzz broker,
25386,_gsl,


## Date Output

In [117]:
# Import the original dataframe
df = pd.read_csv(r"/content/dealer_check.csv", usecols=['dealer_name'], encoding='mac_roman')
df.rename(columns ={'dealer_name':'hand_dealers'}, inplace = True) 

In [118]:
# Concat the original data frame with the results
# dealers = aggregated list of dealers
# hand_dealers = dealers already renamed by hand
# names = cleaned and preprocessed names 
# matches = renamed dealers at 60% accuracy

dealer_checked = pd.concat([dframe2,df,dframe1],
                           axis = 1)

In [120]:
dealer_checked.to_csv('dealer_check_cleaned.csv')